# Example RQ Analysis
----------
This notebook shows how to load the RQ DataFrame generated by `rqpy.process.rq` and recommends some useful functions.

In [1]:
import numpy as np
import qetpy as qp
import rqpy as rp
import pandas as pd
from glob import glob
import pickle as pkl
import matplotlib.pyplot as plt

## Load RQ DataFrame

We specify paths to where the RQs and the raw data are, as well as the channels and corresponding detectors that will be analyzed.

In [2]:
pathtorq = "/path/to/rqs/"
pathtodata = "/path/to/raw/data/"
fs = 625e3

channels = ['PFS1']
det = ['Z1'] * len(channels)

In [3]:
filelist = sorted(glob(f"{pathtorq}*dF*.pkl"))
print(len(filelist))

1000


In [4]:
rq = pd.concat([pd.read_pickle(f) for f in filelist], 
               ignore_index=True)

print(f"Series numbers included: {sorted(set(rq.seriesnumber))}")
print(f"The RQ DataFrame uses {rq.memory_usage(index=True, deep=True).sum()*1e-9:.2f} GB of RAM")

Series numbers included: [11111111111]
The RQ DataFrame uses 0.42 GB of RAM


## Load Template and Look at PSDs

In [5]:
pathtotemplate = "/path/save/template/template_11111111_1111_v1.pkl"
with open(f'{pathtotemplate}', 'rb') as file:
    template_dict = pkl.load(file)

fig, ax = plt.subplots(figsize=(10, 6))
    
for ch, d in zip(channels, det):
    res = qp.ofamp(template_dict[f"template_{ch}{d}"], template_dict[f"template_{ch}{d}"], 
                   template_dict[f"psd_{ch}{d}"], fs, withdelay=False, lgcsigma=True)[-1]
    
    template_dict[f"res_{ch}{d}"] = res
    
    ax.loglog(template_dict[f"f_fold_{ch}{d}"], template_dict[f"psd_fold_{ch}{d}"]**0.5, label=f"{ch}{d}")
    
ax.grid()
ax.grid(which="minor", linestyle="dotted")
ax.tick_params(which="both", direction="in", right=True, top=True)
ax.set_ylim(1e-12, 1e-8)
ax.legend(loc="lower left");
ax.set_title("Template PSDs")
fig.tight_layout()

## Make Initial Basic Cuts

Make some initial cuts based on trigger type and series number. There will likely be more information in the processed RQs that one would want to access, which could be added here.

In [6]:
ctrigger = (rq.triggertype == 1)
crandoms = (rq.triggertype == 3)

cseries = (rq.seriesnumber == 11111111111)

## Recommended Plotting Functions

Here, we list some recommended functions that will help with visualization of the data.

- `rqpy.hist` : a histogram plotting function that allows comparison of arbitrary numbers of cuts on the data
- `rqpy.scatter` : a scatter plotting function for plotting values against each other, allowing arbitrary number of cuts on the data
- `rqpy.densityplot` : a 2D histogram plotting function for visualization of the density of data
- `rqpy.passageplot` : a function for plotting the passage fraction for different cuts as a function of a specified RQ

## Other Recommended Functions

We also list some miscellaneous functions.

- `rqpy.inrange` : a simple function for creating a boolean cut for keeping values within a specified range
- `rqpy.binnedcut` : a function for cutting data based on some dependent variable via 
- `rqpy.make_ideal_template` : a function for creating a double exponential pulse for template making
- `rqpy.shift` : a simple function for shifting values in a 1D array
- `rqpy.fit_gauss` : a function for fitting a Gaussian distribution based on binning the data
- `rqpy.passage_fraction` : a function for calculating the passage fraction of a cut, including uncertainties

## Plotting Raw Data
-----
Here we show an example of using `rqpy.io.getrandevents` for plotting traces randomly pulled from a specified cut.

In [7]:
channels_plot = ["PFS1"]
det_plot = ["Z1"] * len(channels_plot)

series = '111111111_1111'

convtoamps = []
for ch, d in zip(channels_plot, det_plot):
    convtoamps.append(rp.io.get_trace_gain(f"{basepath}{series}/", ch, d)[0])

t, x, cout = rp.io.getrandevents(pathtodata, rq.eventnumber, rq.seriesnumber,
                                 cut=ctrigger, det=det_plot, convtoamps=convtoamps,
                                 channels=channels_plot, fs=fs, indbasepre=16000,
                                 lgcplot=False, ntraces=10, nplot=10, seed=2)